Importing relevant modules and raw data.

In [1]:
from db_utils import QueryAll
import pandas as pd

In [2]:
columns = ["id", "url", "location", "brand", "model", "screenSize", "storage", "postage", "shippingType", 
          "endTime", "returnsAccepted", "returnsDays", "condition", "sellerFeedbackScore", "sellerPositivePercent", 
          "sellerName", "price", "bids", "ram", "resolution", "sellerPositive", "sellerNegative", "quantity", 
          "sellerItemsSold"]
data = pd.DataFrame(QueryAll())
data.columns = columns

The ```isPolice``` column identifies whether a listing is sold by a police auctioneer.

In [3]:
data["isPolice"] = (data.sellerName == "leicester_police_property_disposa...") | (data.sellerName == "sussexpolice-auctions")
data.isPolice = data.isPolice.astype(int)

The ```ram``` column specifies the memory of the device sold, in gigabytes. Any listings with recorded RAM higher than 10 GB are treated as missing values, as those sellers mistakenly thought memory meant storage.

In [4]:
data.ram = data.ram.astype(float)
data.loc[data.ram > 10, "ram"] = -1

The ```storage``` column specifies the storage of the device sold, in gigabytes. Here it is casted to integers.

In [5]:
data.storage = data.storage.astype(int)

The ```sellerLifePercent``` column stores the percentage of positive feedback received by the seller of a listing throughout their time on eBay. The ```sellerYearPercent``` stores this percentage restricted to listings sold within the past year.

In [6]:
data.sellerPositive = data.sellerPositive.astype(str).str.replace(",","").astype(int)
data.sellerNegative = data.sellerNegative.astype(str).str.replace(",","").astype(int)

In [7]:
data["sellerLifePercent"] = data.sellerPositivePercent.str.replace("%","").astype(float)
data["sellerYearPercent"] = data.sellerPositive/(data.sellerPositive+data.sellerNegative).astype(float)*100
data.sellerYearPercent = data.sellerYearPercent.round()
data.loc[data.sellerYearPercent.isna(), "sellerYearPercent"] = 0

The ```sellerItemsSold``` column stores the total number of listings sold by the seller.

In [8]:
def repl(num):
    n = num.group(0)
    if "K" in n:
        if "." in n:
            return n.replace(".","").replace("K","") + "00"
        else:
            return n.replace("K","") + "000"
    elif "M" in n:
        if "." in n:
            return n.replace(".","").replace("M","") + "00000"
        else:
            return n.replace("M","") + "000000"
    
data.sellerItemsSold = data.sellerItemsSold.astype(str).str.replace("[0-9,.]+(M|K)", repl, regex=True).astype(int)

The ```postage``` column stores the cost of shipping of the listing. Free postage is recorded as 0.

In [9]:
def transform_p(po):
    p = str(po)
    if p == "Free" or p == "0":
        return 0
    elif p.startswith("£"):
        return p[1:]
    elif p.startswith("EUR"):
        return float(p[4:])*0.88

data.postage = data.postage.apply(transform_p)

The ```isUsed``` column specifies if the device sold is brand new or used. The ```isBroken``` column specifies if it is partly not working. 

In [10]:
data["isUsed"] = (data.condition == "Used") | (data.condition == "For parts or not working") \
    | (data.condition == "Good - Refurbished")
data["isBroken"] = (data.condition == "For parts or not working")
data.isUsed = data.isUsed.astype(int)
data.isBroken = data.isBroken.astype(int)

The ```deliverySpeed``` column specifies the speed of the postage service used by the seller, labeled from 0 to 2, where 0 is the slowest (economy) and 2 is the fastest (express or courier).

In [11]:
def transform_d(de):
    if de == "Economy Delivery" or de == "An Post International" or de =="Free collection in person":
        return 0
    elif de == "Standard Delivery":
        return 1
    elif de == "Express Delivery" or de == "Courier":
        return 2

data["deliverySpeed"] = data.shippingType.apply(transform_d)

The ```startingPrice``` column stores the minimum price set by the seller.

In [12]:
def transform_s(bids):
    b = eval(bids)
    _b = b[-1].replace(",","")
    if _b.startswith("£"):
        return b[-1][1:]
    elif _b.startswith("EUR"):
        return float(b[-1][4:])*0.88
    
data["startingPrice"] = data.bids.apply(transform_s)

The ```bids``` column contains arrays of highest bids made by distinct bidders, in descending order. It contains the winning price but not the minimum price.

In [13]:
def transform_b(bids):
    b = eval(bids)[:-1]
    _bids = []
    for bid in b:
        _bid = bid.replace(",","")
        if bid.startswith("£"):
            _bids.append(_bid[1:])
        elif bid.startswith("EUR"):
            _bids.append(float(_bid[4:])*0.88)
    return _bids

data.bids = data.bids.apply(transform_b)

The ```isSold``` column indicates whether a listing received at least one bid.

In [14]:
data["isSold"] = data.bids.astype(bool).astype(int)

The ```price``` column records the transaction price of the listing. It is not the highest bid but an increment of the second highest, as per eBay rules.

In [15]:
def transform_pr(p):
    _p = p.replace(",","")
    if p.startswith("£"):
        return _p[1:]
    elif p.startswith("EUR"):
        return float(_p[4:])*0.88

data.price = data.price.astype(str).apply(transform_pr)

Here, all price related columns are adjusted for the quantity of devices sold within each listing.

In [16]:
def transform_q(row):
    row.bids = [round(float(bid)/row.quantity, 2) for bid in row.bids]
    row.startingPrice = round(float(row.startingPrice)/row.quantity, 2)
    row.price = round(float(row.price)/row.quantity, 2)
    return row
    
data = data.apply(transform_q, axis=1)

The ```brand``` and ```model``` columns contain specifications of the device, and here all whitespaces are removed and text made lower case.

In [17]:
data.brand = data.brand.str.lower().str.strip()
data.model = data.model.str.lower().str.strip()

The data is exported as a ```.csv``` file for further analysis.

In [18]:
keep = ["id", "brand", "model", "screenSize", "storage", "postage", "endTime", "returnsAccepted", "returnsDays", 
           "sellerFeedbackScore", "bids", "price", "ram", "resolution", "sellerItemsSold", "isPolice", "sellerLifePercent",
           "sellerYearPercent", "isUsed", "isBroken", "deliverySpeed", "startingPrice", "isSold"]
data = data[keep]

In [19]:
data.to_csv("listings.csv", index=False)

In [20]:
with pd.option_context("display.max_rows", None):
    print(data.model.value_counts())

ipad pro 5                   51
kindle fire 7                45
kindle fire hd 10            35
ipad pro 4                   28
ipad pro 2                   21
ipad air 4                   19
kindle fire 5                18
ipad pro 6                   17
kindle fire hd 7             16
kindle fire 9                15
kindle fire hd 12            13
galaxy tab s2 t813           13
kindle fire kids 7           13
galaxy tab a6 t585           11
galaxy tab a t580            11
galaxy tab a t590             9
galaxy tab a t585             8
kindle paperwhite 7           8
galaxy tab a6 t580            7
ipad pro 1                    7
kindle fire hd 8              6
galaxy tab a8 t290            6
galaxy tab s2 t810            6
kindle fire hd 5              5
kindle basic 7                5
galaxy tab a t595             5
kindle fire hd kids 10        5
galaxy tab s3 t820            5
ipad pro 3                    5
galaxy tab sm-t365            5
ipad 1                        5
kindle f

In [24]:
MODEL = "kindle fire hd kids 10"
print(data[data.model == MODEL].isPolice.value_counts())
data[data.model == MODEL].groupby(["isPolice"]).mean()

0    5
Name: isPolice, dtype: int64


,id,screenSize,storage,returnsAccepted,returnsDays,sellerFeedbackScore,price,ram,sellerItemsSold,sellerLifePercent,sellerYearPercent,isUsed,isBroken,deliverySpeed,startingPrice,isSold
isPolice,,,,,,,,,,,,,,,,
0,2.271067e+11,8.42,32.0,0.4,8.2,465.4,52.2,0.8,287328.2,99.56,99.2,1.0,0.0,1.0,13.534,1.0
